In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops 

from collections import Counter 
import pandas as pd
import numpy as np
import string
import re
from _operator import index
import time
import os

In [2]:
# read processed data
data_X = pd.read_csv('process-dataset/train_X.csv',dtype=object)
data_Y = pd.read_csv('process-dataset/train_Y.csv',dtype=object)

In [3]:
# read X and Y 
train_X = data_X['article']
train_Y = data_Y['tags']

In [4]:
# data_X['article'][4]

In [5]:
train_X = train_X.astype(str)
train_Y = train_Y.astype(str)

In [11]:
train_X[35487]

'data columns id data actual create date sample data id data actual create date 111 open 20180228 111 comment 20180228 111 resolved 20180301 222 open 20180302 222 comment 20180303 222 resolved 20180305 need within single id eg 111 need create column called end date takes actual create date next row makes end date previous entry unless entry resolved end date create date data looks like following id data actual create date end date 111 open 20180228 20180228 111 comment 20180228 20180301 111 resolved 20180301 20180301 222 open 20180302 20180303 222 comment 20180303 20180305 222 resolved 20180305 20180305 apparently need provide queries run far according strawberry select aissueid id bshortid atitle issue_description arequest_type asite bpath b data b actual create date sim_fe left join sim_fe_audit_data b bissueid aissueid b data open comment audit feasibility wouldeep dive scoping pending others awaiting requester info aresolved arequest_type like capacity gets data need unique key tho

In [27]:
def clean_text(text):
    text = re.sub(r'(\s\d+\s)',' ',text)
    text = re.sub(r'[0-9]*','',text)
    
    text = text.split()
    text = " ".join([w for w in text if len(w) >= 2])
    
    return text

In [28]:
train_X = train_X.apply(clean_text)

In [14]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import *

from sklearn.manifold import TSNE

Using TensorFlow backend.


In [32]:
temp = clean_text(train_X[35487])

In [35]:
len(temp.split())

290

In [15]:
### Create X sequence
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_X)

In [20]:
sequences = tokenizer.texts_to_sequences(train_X)
seq_x = pad_sequences(sequences,maxlen=150)

In [24]:
seq_x[35487]

array([   3,  195,  391,  118,   22,  329,  131,   13, 6307,  554,   88,
        170, 1640,   86,    7, 9893,  727,    7,   86, 2184,   60,  639,
         96, 1180,  244,  312,  136,   96,  722,   60,   96,  566, 1532,
       2611, 1136,  214,  566,  722,   60,   96,  698,  566, 3418,   96,
         69, 6586,  364,  232,   38,  288,   64, 2546,  262,  271,  375,
         86,  355,  214,   93,   53,    3,  195,  391, 6677,   22,   57,
         13,   57,  722,   60,   96,   57,   47,   96, 6138, 4059,  170,
        394,   53,   13,  722,   60,   96, 2243,  121,    3,   59, 2243,
         47, 2243,    3,   47,   96,   53, 2243,  192,   53,    3,   53,
       6138, 4059, 4943,   13,   13,  194,  654, 4943, 2907, 1240,  199,
        173,  722,   60,   96,  776,  394,   53,    3,  722,   60,   96,
         47,   96, 6138, 4059, 4943,   13, 2465,   57,  118,   22,   14,
       3632,   30, 1334,  614,   47, 1279, 4328, 5955, 1628,  271, 1983,
        271,  394,  185, 2862,  135,   32,   57])

In [25]:
dic = tokenizer.word_index

In [26]:
for word,index in dic.items():
    for s in seq_x[35487]:
        if s == index:
            print(word + " " + str(index))

tried 69
audit 4943
audit 4943
audit 4943
issue 195
issue 195
r 192
id 3
id 3
id 3
id 3
id 3
id 3
stop 639
one 30
thus 1532
think 288
unique 727
arequest 6677
path 131
description 391
description 391
step 566
step 566
step 566
rank 2243
rank 2243
rank 2243
rank 2243
site 329
pending 2907
capacity 3632
way 38
order 173
strange 1334
info 199
could 88
rows 271
rows 271
rows 271
http 32
request 118
request 118
tid 2465
every 232
answer 364
k 355
data 13
data 13
data 13
data 13
data 13
data 13
visualization 6307
causes 1628
join 394
join 394
join 394
repeatedly 5955
others 1240
end 47
end 47
end 47
end 47
end 47
process 244
actual 722
actual 722
actual 722
actual 722
actual 722
actual 722
like 14
hopefully 2862
become 1983
basically 614
next 214
next 214
ending 3418
open 194
type 22
type 22
type 22
duplicated 4328
fe 4059
fe 4059
fe 4059
base 375
currently 312
date 96
date 96
date 96
date 96
date 96
date 96
date 96
date 96
date 96
date 96
date 96
date 96
work 64
previous 698
query 86
query 

In [15]:
train_X[1]

'code produc thisselect end date exhibit room placeofexhibit end date gtcurrent date minimum andr maximum locat name name exhibit orderbi end date desc column row requir second biggest one alway need second biggest one though depend count room room queri return need fourth biggest one requir get subquerythi subqueri haveselect end date exhibit room placeofexhibit end date gtcurrent date minimum andr maximum locat name name exhibit orderbi end date asc offset select count name room minimum maximum limit return queri get second last date wrong direct get second smallest instead second largest'

In [16]:
seq_x[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    9,  893,   96,   95, 9678, 1290,   96,
         95, 7200,   95, 1760, 1427,  194,    3,    3, 9678, 2680,   96,
         95, 1046,  102,   66,  153,  213, 5082,   32,  479,   39,  213,
       5082,   32,  699,  311,  170, 1290, 1290,   93,   15,   39, 3775,
       5082,   32,  153,    5, 3451,   96,   95, 9678, 1290,   96,   95,
       7200,   95, 1760, 1427,  194,    3,    3, 9678, 2680,   96,   95,
       1831,  842,   45,  170,    3, 1290, 1760, 1427,  625,   15,   93,
          5,  213,  262,   95,  243,  344,    5,  213, 4258,  360,  213,
       3549])

In [17]:
for word,index in dic.items():
    for s in seq_x[1]:
        if s == index:
            print(word + " " + str(index))

maximum 1427
maximum 1427
maximum 1427
desc 1046
row 66
date 95
date 95
date 95
date 95
date 95
date 95
date 95
date 95
date 95
name 3
name 3
name 3
name 3
name 3
last 262
smallest 4258
direct 344
room 1290
room 1290
room 1290
room 1290
room 1290
count 170
count 170
return 15
return 15
limit 625
requir 153
requir 153
code 9
select 45
end 96
end 96
end 96
end 96
end 96
end 96
need 39
need 39
wrong 243
alway 479
depend 311
biggest 5082
biggest 5082
biggest 5082
locat 194
locat 194
column 102
though 699
instead 360
get 5
get 5
get 5
produc 893
largest 3549
queri 93
queri 93
fourth 3775
exhibit 9678
exhibit 9678
exhibit 9678
exhibit 9678
one 32
one 32
one 32
orderbi 2680
orderbi 2680
gtcurrent 7200
gtcurrent 7200
second 213
second 213
second 213
second 213
second 213
asc 1831
offset 842
minimum 1760
minimum 1760
minimum 1760
subqueri 3451
